In [4]:
import numpy as np
import pandas as pb
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import udf, col, when
from IPython.display import Image
from IPython.display import display

In [5]:
spark = SparkSession.builder.appName("Recommendation Movie").getOrCreate()

23/10/31 08:31:03 WARN Utils: Your hostname, mason-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/10/31 08:31:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/31 08:31:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
sc = spark.sparkContext
sqlContext = SQLContext(sc)

/usr/local/lib/python3.8/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [7]:
ratings_df = spark.read.csv("rating.csv", inferSchema= True, header=True)

In [8]:
ratings_df = ratings_df.drop('timestamp')

In [9]:
movies_df = spark.read.csv("movie.csv", inferSchema= True, header=True)

In [10]:
links_df = spark.read.csv("link.csv", inferSchema= True, header=True)

In [11]:
traning_df, validation_df = ratings_df.randomSplit([0.8,0.2])

In [12]:
interations = 10
regularization_parameter = 0.1
rank = 4
errors = []
err = 0

In [13]:
als = ALS(maxIter= interations, regParam=regularization_parameter, rank=4, userCol="userId", itemCol="movieId", ratingCol = "rating")
model = als.fit(traning_df)

23/10/31 08:33:38 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/10/31 08:33:42 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [14]:
predictions = model.transform(validation_df)
new_predictions = predictions.filter(col('prediction') != np.nan)
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')
rmse = evaluator.evaluate(new_predictions)

In [15]:
print('Erorror: '+str(rmse))

Erorror: 0.8207114183544837


In [16]:
predictions.show(n=10)

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|     1|     29|   3.5|  3.610647|
|     1|    293|   4.0| 3.9503818|
|     1|    318|   4.0|  4.242936|
|     1|    593|   3.5|  3.980338|
|     1|   1217|   3.5| 3.6358538|
|     1|   1266|   4.0| 3.7535038|
|     1|   1333|   4.0|  3.540858|
|     1|   1525|   3.0| 2.8079455|
|     1|   1848|   3.5| 3.0699472|
|     1|   2118|   4.0| 3.5903404|
+------+-------+------+----------+
only showing top 10 rows



In [1]:
predictions.join(movies_df, "movieId").select("userId", "title", "genres", "prediction").show(5)

NameError: name 'predictions' is not defined

RECOMMEND FOR USER

In [23]:
for_one_user = predictions.filter(col("userId")==599).join(movies_df, "movieId").join(links_df, "movieId").select("userId", "title", "genres","tmdbId" ,"prediction")
for_one_user.show(5)

+------+--------------------+--------------------+------+----------+
|userId|               title|              genres|tmdbId|prediction|
+------+--------------------+--------------------+------+----------+
|   599|Naked Gun 33 1/3:...|       Action|Comedy| 36593| 2.8472595|
|   599|   Striptease (1996)|        Comedy|Crime|  9879| 2.3792806|
|   599|Eyes Wide Shut (1...|Drama|Mystery|Thr...|   345| 2.6640959|
|   599|Matchstick Men (2...|  Comedy|Crime|Drama|  7270|  3.145326|
|   599|Secondhand Lions ...|Children|Comedy|D...| 13156|  3.105628|
+------+--------------------+--------------------+------+----------+



In [25]:
import webbrowser
link = "https://www.themoviedb.org/movie/"
for movie in for_one_user.take(2):
    movieURL = link+str(movie.tmdbId)
    print(movie.title)
    webbrowser.open(movieURL)

Naked Gun 33 1/3: The Final Insult (1994)
Striptease (1996)


In [ ]:
for_one_user8 = predictions.filter(col("userId")==9).join(movies_df, "movieId").join(links_df, "movieId").select("userId", "title", "genres","tmdbId" ,"prediction")
for_one_user8.show(5)

In [ ]:
import webbrowser
link = "https://www.themoviedb.org/movie/"
for movie in for_one_user8.take(2):
    movieURL = link+str(movie.tmdbId)
    print(movie.title)
    webbrowser.open(movieURL)

In [26]:
userRecommends = model.recommendForAllUsers(5)
movieRecommends = model.recommendForAllItems(5)

In [27]:
userRecommends.select("userId", "recommendations.movieId").show(10, False)

+------+----------------------------------------+
|userId|movieId                                 |
+------+----------------------------------------+
|1     |[121029, 95776, 128981, 120821, 77736]  |
|3     |[121029, 95776, 120821, 128981, 120134] |
|5     |[121029, 120821, 77736, 95776, 126219]  |
|6     |[121029, 126219, 107434, 107252, 107559]|
|9     |[121029, 107306, 107434, 107252, 107559]|
|12    |[121029, 120821, 95776, 73529, 120134]  |
|13    |[121029, 112577, 128981, 95595, 107306] |
|15    |[126219, 121029, 107559, 120821, 95776] |
|16    |[121029, 120821, 95776, 104780, 128981] |
|17    |[121029, 128981, 107306, 77736, 107434] |
+------+----------------------------------------+
only showing top 10 rows



In [ ]:
movieRecommends.select("movieId", "recommendations.userId").show(10, False)

In [ ]:
user = ratings_df.select("userId").distinct().limit(3)
user.show()

In [ ]:
userSubsetRecs = model.recommendForUserSubset(users,10)
userSubsetRecs.show()

In [ ]:
userSubsetRecs.select("userId", "recommendations.movieId").show(10, False)

In [ ]:
user = ratings_df.select("movieId").distinct().limit(3)
user.show()

In [ ]:
movieSubsetRecs = model.recommendForUserSubset(movies,10)
movieSubsetRecs.select("movieId", "recommendations.userId").show(10, False)

In [ ]:
movie_ids = [1580,3175,2366,1590]
user_ids = [543,543,543,543]
new_users_pred = sqlCOntext.createDataFrame(zip(movie_ids, user_ids), schema=['movieId', 'userId'])

In [ ]:
new_predictions = model.transform(new_users_pred)
new_predictions.show()